In [1]:
import json, boto3, logging
from botocore.exceptions import ClientError

In [42]:
with open("./.KEYS/API_KEYS.json", "r") as f:
    key = json.load(f)

# S3 버킷 정보 get
with open("./.KEYS/DATA_SRC_INFO.json", "r") as f:
    bucket_info = json.load(f)

with open("./.KEYS/GEMINI_API_KEY.json", "r") as f:
    gemini_api_key = json.load(f)

In [3]:
dynamo_table_name = bucket_info['restore_table_name']

In [4]:
dynamodb = boto3.resource(
        'dynamodb',
        aws_access_key_id=key['aws_access_key_id'],
        aws_secret_access_key=key['aws_secret_key'],
        region_name=key['region']
    )
table = dynamodb.Table(dynamo_table_name)

In [5]:
scan_kwargs = {}
data = []

In [6]:
# Pagination을 고려하여 모든 데이터를 가져오기
while True:
    response = table.scan(**scan_kwargs)
    data.extend(response.get('Items', []))
    scan_kwargs['ExclusiveStartKey'] = response.get('LastEvaluatedKey', None)
    if not scan_kwargs['ExclusiveStartKey']:
        break

In [7]:
wanted_data = [obj for obj in data if obj.get('site_symbol', '').upper() == "WAN"]
jobkorea_data = [obj for obj in data if obj.get('site_symbol', '').upper() == "JK"]

In [8]:
str(wanted_data[5])

"{'company_id': Decimal('18028'), 'site_symbol': 'WAN', 'job_title': '정보보안 담당자', 'job_prefer': '정보보호 관련 학과 및 IT 관련학 전공 클라우드 보안솔루션 운영 경험이 있으신 분 인프라 취약점 점검 및 웹 모의해킹 수행 가능하신 분 ISO27701, ISMS-P 보안인증 대응 경험이 있으신 분 ISMS-P 인증심사원, CPPG, CISA, CISSP, 정보보안기사 자격증을 보유하신 분', 'crawl_url': 'https://www.wanted.co.kr/wd/230608', 'end_date': None, 'job_id': Decimal('230608'), 'crawl_domain': 'www.wanted.co.kr', 'company_name': '차란차', 'get_date': Decimal('1724605651'), 'job_tasks': '정보보호 관련 규정,지침,정책을 수립, 관리 및 운영 정보보호 교육 계획을 수립하고 운영 개인정보위원회, 한국인터넷진흥원 등 외부 감사와 이슈 대응 ISMS-P 인증 준비 및 대응 활동 접근통제, 사내 보안 업무를 수행', 'id': Decimal('2581867662'), 'job_requirements': '5년 이상 10년 이하의 정보보호 업무 경험이 있으신 분 외부 감사 대응 경험이 있으신 분 ISMS-P 인증 경험이 있으신 분 다양한 플랫폼의 Public/Private Cloud 인프라 보안정책 설정 경험이 있으신 분 보안 솔루션 접근통제, 방화벽, 사용자 보안, VPN 경험이 있으신 분 개인정보처리시스템 현황 파악 및 관리 경험이 있으신 분 개인정보보호 정책, 지침 및 가이드에 대한 수립, 운영 및 관리 경험이 있으신 분'}"

In [9]:
str(jobkorea_data[3])

"{'indurstry_type': '소프트웨어 자문,개발,공급/컴퓨터주변기기 도소매', 'site_symbol': 'JK', 'job_title': '광화문 흥국생명 차세대시스템 구축 Java, BXM 프레임웍, 융자업무 관련 경험자를 모집 합니다. (s01)', 'job_category': '응용 소프트웨어 개발 및 공급업', 'crawl_url': 'https://www.jobkorea.co.kr/Recruit/GI_Read/45423573', 'end_date': '1724598000', 'required_career': False, 'job_id': '45423573', 'company_name': '행복추구 패러다임㈜kcms', 'resume_required': True, 'start_date': '1724511600', 'get_date': Decimal('1724605790'), 'job_tasks': '광화문 흥국생명 차세대시스템 구축', 'id': Decimal('2352333735'), 'stacks': 'JAVA, BXM 프레임웍, 융자업무 관련 경험자'}"

In [29]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai
"""

import google.generativeai as genai

#genai.configure(api_key=os.environ["GEMINI_API_KEY"])
genai.configure(api_key=gemini_api_key['GEMINI_API'])

# Create the model
generation_config = {
  "temperature": 0.7,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[
  ]
)

In [38]:
wanted_data[6]

{'company_id': Decimal('3260'),
 'site_symbol': 'WAN',
 'job_title': 'QA 엔지니어',
 'job_prefer': 'CSTE, ISTQB, CSTS 등 자격증 보유자 성능테스트, 자동화테스트 경험자 개발 경력 보유자 CI/CD에 대한 이해와 경험 우대 Jenkins 등',
 'crawl_url': 'https://www.wanted.co.kr/wd/19232',
 'end_date': None,
 'job_id': Decimal('19232'),
 'crawl_domain': 'www.wanted.co.kr',
 'company_name': '모노리스',
 'get_date': Decimal('1724605651'),
 'job_tasks': 'Mobile App, Mobile Web, Back-office, 예약,판매관리시스템 등 모노리스에서 운영하는 서비스의 전반적인 품질 관리 서비스를 구성하는 전장, 트랙 센서 등 IoT 기기들에 적용되는 소프트웨어 품질 관리 소프트웨어 요구사항 및 구성에 대한 리뷰 피드백 결함 보고, 추적 및 분석에 따른 이슈 관리 개발 프로세스에서 다양한 기술이나 기법을 적용 더 좋은 개발문화를 함께 만들기',
 'id': Decimal('434466414'),
 'job_requirements': '5년 이상의 QA 업무 경험 iOS/Android Mobile Application QA 경험자 컴퓨터 관련 학과 졸업 or 그에 상응하는 역량 보유 빠른 프로세스에 유연하게 대응이 가능하신 분'}

In [43]:
# WANTED propmt

tech_stack_list = ["programming languages", "frameworks", "databases", "cloud services", "devops tools", "version control tools", "ci cd tools", "testing tools", "package management tools", "api management tools"]
info_keys = ["job_title", "job_prefer", "job_task", "job_requirements"]
info_keys_str = ', '.join(info_keys)
tech_stack_list_str = ', '.join(tech_stack_list)
input_data = str(wanted_data[6])
input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.\n- `dev_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 프로그램 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.\n- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.\n[Input Data]\n{input_data}"
#input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 포맷의 텍스트 데이터에서 {info_keys_str} key의 value 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요.\n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 해당 key의 value는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data에서 유추한 직무의 산업 분류입니다. value 값은 영단어로 이루어진 list여야 합니다.\n- `required_career`: Input Data에서 경력을 요구유무를 나타냅니다. 경력일 경우 `True` BOOLEAN값을, 신입일 경우 `False` BOOLEAN값을 넣어주세요.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

```json
{
  "dev_stack": [
    "mobile app testing",
    "mobile web testing",
    "performance testing",
    "automation testing",
    "CI/CD",
    "Jenkins",
    "IoT",
    "software requirements",
    "defect tracking",
    "issue management"
  ],
  "job_requirements": [
    "QA 업무 경험 5년 이상",
    "iOS/Android Mobile Application QA 경험",
    "컴퓨터 관련 학과 졸업 또는 그에 상응하는 역량 보유",
    "빠른 프로세스에 유연하게 대응 가능"
  ],
  "job_prefer": [
    "CSTE, ISTQB, CSTS 등 자격증 보유",
    "성능테스트, 자동화테스트 경험",
    "개발 경력 보유",
    "CI/CD에 대한 이해와 경험",
    "Jenkins 사용 경험"
  ],
  "job_category": [
    "QA Engineer",
    "Software Quality Assurance",
    "Testing"
  ],
  "indurstry_type": [
    "IT",
    "Software",
    "Technology",
    "Mobile",
    "IoT"
  ],
  "required_career": true
}
```


In [35]:
# jobkorea propmt

info_keys = ["indurstry_type", "job_title", "job_category", "job_tasks", "stacks"]
info_keys_str = ', '.join(info_keys)
input_data = str(jobkorea_data[3])
input_text = f"[문맥]\n당신은 IT-Software 전문 도메인을 가진 데이터 전문가입니다. 주어진 json형태의 데이터에서 필요한 데이터를 추출하거나 요약하는 임무를 맡았습니다.\n[Instruction]\nInput Data에서 주어진 json 데이터에서 {info_keys_str} key의 값을 바탕으로 아래에 주어진 key 값들에 대한 value 값을 채워주세요. 단, 모를 경우 \"null\"을 넣어주세요. \n- `tech_stack`: Input Data를 기반으로 유추하거나 추출한 개발에 사용되는 기술의 모음입니다. 결과는 항상 영단어로 이루어진 list여야 합니다.\n- `job_requirements`: Input data를 기반으로 유추한 직무의 요구사항입니다. 결과는 한글로 된 열 개 단어 이하인 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_prefer`: Input data를 기반으로 유추한 직무의 우대사항입니다. 결과는 한글로 된 열 개 단어 이하의 문장 또는 단어이며, list 포맷으로 제시됩니다.\n- `job_category`: Input Data를 기반으로 유추한 직무(position) 분류 입니다. 결과는 영단어로 이루어진 list여야 합니다.\n- `indurstry_type`: Input Data의 직무의 산업 분류입니다. 결과는 영단어로 이루어진 list여야 합니다.\n[Input Data]\n{input_data}"
response = chat_session.send_message(input_text)

print(response.text)

```json
{
  "tech_stack": [
    "Java",
    "BXM Framework"
  ],
  "job_requirements": [
    "융자업무 관련 경험"
  ],
  "job_prefer": [
    "null"
  ],
  "job_category": [
    "Software Development",
    "Application Development"
  ],
  "indurstry_type": [
    "Software Development",
    "IT Consulting",
    "Financial Technology"
  ]
}
```
